In [31]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import re

In [32]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [33]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [34]:
%%capture pwd
!pwd

In [35]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('is-the-uk-on-track-to-meet-its-climate-commitments', 300, 500)

# Fig 1

In [36]:
df = pd.read_excel("raw/EO_Article_-_UK_targets_Fig_1.xlsx", sheet_name="Graph")
df = df.set_index("Unnamed: 0").T.drop("2030 Target", axis=1).reset_index()
df.columns = ["year", "historic", "projected"]

In [37]:
f = "fig1_projected-emissions"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,historic,projected
0,1990,793.78739,NaN
1,1991,802.64404,NaN
2,1992,781.03397,NaN
3,1993,761.13853,NaN
4,1994,751.96395,NaN


In [38]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="⬅ historical   projection ➡",
            titleY=-15,
            titleX=302,
            titleFontWeight="normal",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            titleFontSize=10,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
line = base.mark_line(color=colors["eco-dark-blue"]).encode(
    y=alt.Y(
        "historic:Q",
        axis=alt.Axis(
            grid=True,
            gridOpacity=0.1,
            gridColor=colors["eco-gray"],
            title="GtCO₂e per year",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=25,
            titleY=28,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
    )
).transform_filter('datum.year<=2020')
area = base.mark_line(color=colors["eco-turquiose"]).encode(y=alt.Y("projected:Q")).transform_filter('datum.year>=2020')
ruler = (
    base.mark_line(color=colors["eco-gray"])
    .encode(y="y:Q")
    .transform_calculate(y="254")
)
label = (
    alt.Chart(pd.DataFrame([{"x": 2000, "y": 280, "t": "2030 target"}]))
    .mark_text(color=colors["eco-gray"],fontSize=10)
    .encode(text="t:N", x="x:Q", y="y:Q")
)
r2020 = (
    base.mark_rule(
        color=scale_lightness(colors["eco-gray"], 1.5),
        strokeDash=[5, 5],
        strokeWidth=0.5,
    )
    .encode(x="x:Q")
    .transform_calculate(x="2020")
)
title = alt.TitleParams(
    "UK emissions up to date and projected reduction to 2030",
    subtitle=["Source: BEIS (2020)"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    ((ruler + r2020 + area + line + label).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "w").write(readme)
layer1

alt.LayerChart(...)

In [39]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="⬅ historical   projection ➡",
            titleY=-15,
            titleX=302,
            titleFontSize=12,
            labelFontSize=12,
            titleFontWeight="normal",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            format=".0f",
            # zindex=1,
            # offset=-43
        ),
    )
)
line = base.mark_line(color=colors["eco-mid-blue"]).encode(
    y=alt.Y(
        "historic:Q",
        axis=alt.Axis(
            gridOpacity=0.1,
            gridColor=colors["eco-gray"],
            title="GtCO₂e per year",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=12,
            labelFontSize=12,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=31,
            titleY=30.5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
    )
).transform_filter('datum.year<=2020')
area = base.mark_line(color=colors["eco-turquiose"]).encode(y=alt.Y("projected:Q")).transform_filter('datum.year>=2020')
ruler = (
    base.mark_line(color=colors["eco-gray"])
    .encode(y="y:Q")
    .transform_calculate(y="254")
)
label = (
    alt.Chart(pd.DataFrame([{"x": 2000, "y": 290, "t": "2030 target"}]))
    .mark_text(color=colors["eco-gray"],fontSize=12)
    .encode(text="t:N", x="x:Q", y="y:Q")
)
r2020 = (
    base.mark_rule(
        color=scale_lightness(colors["eco-gray"], 0.7),
        strokeDash=[5, 5],
        strokeWidth=0.5,
    )
    .encode(x="x:Q")
    .transform_calculate(x="2020")
)
title = alt.TitleParams(
    "UK emissions up to date and projected reduction to 2030",
    subtitle=["Source: BEIS (2020)"],
    anchor="start",
    align="left",
    dx=5,
    fontSize=14,
    subtitleFontSize=12,
    color=colors["eco-dot"],
    subtitleColor=colors["eco-dot"],
    subtitleFontStyle="italic",
)
layer1 = (
    ((ruler + r2020 + area + line + label).properties(height=300, width=400))
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + "_dark.json")
layer1.save("visualisation/" + f + "_dark.png", scale_factor=2.0)
layer1.save("visualisation/" + f + "_dark.svg")
readme=re.sub(f, f+'_dark', readme)
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)